# Merging Communities
There are n people numbered from 0 to n - 1 , with each person initially belonging to a separate community. When two people from different communities connect, their communities merge into a single community.

Your goal is to write two functions:
- connect(x: int, y: int) -> None: Connects person x with person y and merges their communities.
- get_community_size(x: int) -> int: Returns the size of the community which person x belongs to.

**Example:**
```python
Input: n = 5,
       [
         connect(0, 1),
         connect(1, 2),
         get_community_size(3),
         get_community_size(0),
         connect(3, 4),
         get_community_size(4),
       ]
Output: [1, 3, 2]
```

## Intuition

In this problem, we start with **n** individuals, each in their own separate community. As we connect pairs of individuals, their respective communities merge. The challenge is to efficiently manage these connections and quickly determine the size of the community for any given individual.

This is where the **Union-Find** data structure, also known as the **Disjoint Set Union (DSU)**, comes in. Union-Find supports two main operations:

- **Union**: Takes two elements from different sets and merges them into the same set.
- **Find**: Determines the representative (or root) of the set an element belongs to.

In the context of this problem, the **union** operation is used to merge the communities of two individuals, while the **find** operation helps determine which community a person belongs to. To implement this, we designate a **representative** for each community and represent these communities as a **graph**, where each community is a connected component, and each person points to their representative.

This can be efficiently managed using a **parent array**, where `parent[i]` stores the parent of person `i`.

---

## Union

Before connecting two people, we first need to identify their representatives using the **find** function.

Once we identify their respective communities, we need to decide which representative will lead the merged community. One approach is to attach all members of one community to the representative of the other. In code, this is done by updating:

```python
parent[rep_y] = rep_x;
```

This ensures that all members of the second community are now indirectly connected to the new representative.

---

### Find

The **find** function returns the representative (or root) of the community a person belongs to.

#### Difference Between a Parent and a Representative:
- A **representative** is the root of a community and serves as its leader.
- A **parent** is the immediate node that a person points to in the union-find structure.

To determine the representative of a person, we repeatedly follow the parent chain until we reach a node that points to itself, which signifies the root.

---

### Union by Size

Before merging two communities, we must decide which representative will lead the merged community. The key idea is to **minimize the depth of the tree** by always attaching the smaller community to the larger one. This prevents the tree from growing too deep and helps maintain efficient operations.

To achieve this, we keep track of the size of each community. When merging two communities, the representative of the smaller community is linked to the representative of the larger community. This ensures that the overall tree structure remains balanced.

---

### Path Compression

A significant optimization to the **find** operation is **path compression**. Instead of keeping the original structure of the tree, every node in the search path is directly linked to the root of the community. 

#### How Path Compression Works:
- Each time we perform a **find** operation, we traverse up the tree to find the representative.
- Rather than leaving the intermediate nodes unchanged, we update all nodes along the path to point directly to the root.
- This flattening of the tree drastically reduces the depth of the structure, making future operations significantly faster.

---

### Complexity Analysis

#### Time Complexity
With **path compression** and **union by size**, the operations become highly efficient:
- **Find**: Runs in **amortized O(1)** because the tree structure becomes extremely flat over time.
- **Union**: Also runs in **amortized O(1)** since it only requires two **find** operations.
- **Get Size**: Runs in **amortized O(1)** as it only requires a single **find** operation.

Overall, both **connect** and **get_community_size** have an **amortized O(1) complexity**, while the constructor runs in **O(n)** due to the initialization of the data structures.

#### Space Complexity
The space complexity is **O(n)** since the **parent** and **size** arrays each store information for all individuals. Additionally, since **path compression** significantly reduces recursion depth, the space used by recursive calls in **find** is also **amortized O(1)**.

In [1]:
class UnionFind:
    def __init__(self, size: int):
        self.parent = [i for i in range(size)]
        self.size = [1] * size
    
    def union(self, x: int, y: int) -> None:
        rep_x, rep_y = self.find(x), self.find(y)

        if rep_x != rep_y:
            if self.size[rep_x] > self.size[rep_y]:
                self.parent[rep_y] = rep_x
                self.size[rep_x] += self.size[rep_y]
            else:
                self.parent[rep_x] = rep_y
                self.size[rep_y] += self.size[rep_x]
    
    def find(self, x: int) -> int:
        if x == self.parent[x]:
            return x
        
        self.parent[x] = self.find(self.parent[x])
        return self.parent[x]
    
    def get_size(self, x: int) -> int:
        return self.size[self.find(x)]


class MergingCommunities:
    def __init__(self, n: int):
        self.uf = UnionFind(n)

    def connect(self, x: int, y: int) -> None:
        self.uf.union(x, y)

    def get_community_size(self, x: int) -> int:
        return self.uf.get_size(x)